## Ajouter des comportements aux données de classe avec des propriétés 

Tout au long de ces guide, nous nous sommes concentrés sur la séparation du comportement et des données. C'est très important dans la programmation orientée objet, mais nous sommes sur le point de voir qu'en Python, la distinction est étrangement floue. Python est très bon pour brouiller les distinctions ; cela ne nous aide pas vraiment à sortir des sentiers battus. Au contraire, cela nous apprend à arrêter de penser à la boîte. Avant d'entrer dans les détails, discutons de quelques mauvais principes de conception orientés objet. 

De nombreux développeurs orientés objet nous apprennent à ne jamais accéder directement aux attributs. Ils insistent pour que nous écrivions l'accès aux attributs comme ceci :

In [ ]:
class Color:    
  def __init__(self, rgb_value: int, name: str) -> None:        
    self._rgb_value = rgb_value        
    self._name = name    
    
  def set_name(self, name: str) -> None:        
    self._name = name    
    
  def get_name(self) -> str:        
    return self._name    
    
  def set_rgb_value(self, rgb_value: int) -> None:        
    self._rgb_value = rgb_value    
  
  def get_rgb_value(self) -> int:        
    return self._rgb_value


Les variables d'instance sont précédées d'un trait de soulignement pour suggérer qu'elles sont privées (d'autres langages les forceraient en fait à être privées). Ensuite, les méthodes get et set donnent accès à chaque variable. Cette classe serait utilisée en pratique comme suit :

In [ ]:
c = Color(0xff0000, "bright red")

print(c.get_name())

c.set_name("red")
c.get_name()

bright red


'red'

L'exemple ci-dessus n'est pas aussi lisible que la version à accès direct que Python privilégie

In [ ]:
class Color_Py:    
  def __init__(self, rgb_value: int, name: str) -> None:        
    self.rgb_value = rgb_value        
    self.name = name

Voici comment fonctionne cette classe. c'est un peu plus simple.

In [ ]:
c = Color_Py(0xff0000, "bright red")
print(c.name)
c.name = "red"

print(c.name)

bright red
red


Alors, pourquoi insisterait-on sur la syntaxe basée sur la méthode ? L'idée de setters et getters semble utile pour encapsuler les définitions de classe.

 Certains outils basés sur Java peuvent générer automatiquement tous les getters et setters, les rendant presque invisibles. L'automatisation de leur création n'en fait pas une bonne idée. La raison historique la plus importante pour avoir des getters et des setters était de faire en sorte que la compilation séparée des binaires fonctionne de manière ordonnée. Sans avoir besoin de lier des binaires compilés séparément, cette technique ne s'applique pas toujours à Python. 
 
 Une justification en cours pour les getters et les setters est que, un jour, nous pourrions vouloir ajouter du code supplémentaire lorsqu'une valeur est définie ou récupérée. Par exemple, nous pourrions décider de mettre en cache une valeur pour éviter des calculs complexes, ou nous pourrions vouloir valider qu'une valeur donnée est une entrée appropriée

 Par exemple, nous pourrions décider de changer la méthode set_name() comme suit

In [ ]:
class Color_V:    
  def __init__(self, rgb_value: int, name: str) -> None:        
    self._rgb_value = rgb_value        
    if not name:            
      raise ValueError(f"Invalid name {name!r}")        
      self._name = name    
    
  def set_name(self, name: str) -> None:        
    if not name:            
      raise ValueError(f"Invalid name {name!r}")        
    self._name = name

Si nous avions écrit notre code d'origine pour l'accès direct aux attributs, puis que nous l'avions changé plus tard en une méthode comme la précédente, nous aurions un problème : toute personne ayant écrit du code accédant directement à l'attribut devrait maintenant changer son code en accéder à une méthode. S'ils ne changeaient pas le style d'accès d'un accès aux attributs à un appel de fonction, leur code serait cassé. 

Le mantra selon lequel nous devrions rendre tous les attributs privés, accessibles via des méthodes, n'a pas beaucoup de sens en Python. Le langage Python manque de toute notion réelle de membres privés ! Nous pouvons voir la source; nous disons souvent "Nous sommes tous des adultes ici." Que pouvons-nous faire? Nous pouvons rendre la distinction syntaxique entre attribut et méthode moins visible. 

Python nous donne la fonction de propriété pour créer des méthodes qui ressemblent à des attributs. Nous pouvons donc écrire notre code pour utiliser l'accès direct aux membres, et si nous devons de manière inattendue modifier l'implémentation pour effectuer un calcul lors de l'obtention ou de la définition de la valeur de cet attribut, nous pouvons le faire sans changer l'interface. Voyons à quoi cela ressemble :

In [ ]:
class Color_VP:    
  
  def __init__(self, rgb_value: int, name: str) -> None:        
    self._rgb_value = rgb_value        
    if not name:            
      raise ValueError(f"Invalid name {name!r}")        
    self._name = name    
    
  def _set_name(self, name: str) -> None:        
    if not name:            
      raise ValueError(f"Invalid name {name!r}")
    self._name = name    
  
  def _get_name(self) -> str:        
    return self._name    
  
  name = property(_get_name, _set_name)

Par rapport à la classe précédente, nous changeons d'abord l'attribut name en un attribut _name (semi-)privé. Ensuite, nous ajoutons deux autres méthodes (semi-)privées pour obtenir et définir cette variable, en effectuant notre validation lorsque nous la définissons. 

Enfin, nous avons la construction de la propriété en bas. C'est la magie Python. Il crée un nouvel attribut sur la classe Color appelé name. Il définit cet attribut comme une propriété. Sous le capot, un attribut de propriété délègue le vrai travail aux deux méthodes que nous venons de créer. 

Lorsqu'elle est utilisée dans un contexte d'accès (à droite du = ou :=), la première fonction obtient la valeur. Lorsqu'elle est utilisée dans un contexte de mise à jour (sur le côté gauche de = ou :=), la deuxième fonction définit la valeur. Cette nouvelle version de la classe Color peut être utilisée exactement de la même manière que la version précédente, mais elle effectue maintenant une validation lorsque nous définissons l'attribut name

In [ ]:
c = Color_VP(0xff0000, "bright red")

print(c.name) 
c.name = "red"

print(c.name)

c.name = ""

bright red
red


ValueError: ignored

Ainsi, si nous avions précédemment écrit du code pour accéder à l'attribut name, puis l'avions modifié pour utiliser notre objet basé sur la propriété, le code précédent fonctionnerait toujours. 

S'il tente de définir une valeur de propriété vide, c'est un comportement que nous voulions interdire. Succès ! Gardez à l'esprit que, même avec la propriété name, le code précédent n'est pas sûr à 100 %. Les utilisateurs peuvent toujours accéder directement à l'attribut _name et le définir sur une chaîne vide s'ils le souhaitent. Mais s'ils accèdent à une variable que nous avons explicitement marquée d'un trait de soulignement pour suggérer qu'elle est privée, ce sont eux qui doivent faire face aux conséquences, pas nous. Nous avons établi un contrat formel, et s'ils choisissent de rompre le contrat, ils en assument les conséquences

## Propriétés en détail

Pensez à la fonction de propriété comme renvoyant un objet qui représente toutes les demandes d'obtention ou de définition de la valeur de l'attribut via les noms de méthode que nous avons spécifiés. 

La propriété intégrée est comme un constructeur pour un tel objet, et cet objet est défini comme membre public pour l'attribut donné. Ce constructeur de propriété peut en fait accepter deux arguments supplémentaires, une fonction de suppression et une chaîne de documentation pour la propriété. 

La fonction de suppression est rarement fournie en pratique, mais elle peut être utile pour enregistrer le fait qu'une valeur a été supprimée, ou éventuellement pour opposer son veto à la suppression si nous avons des raisons de le faire. 

La docstring est juste une chaîne décrivant ce que fait la propriété, pas différente des docstrings dont nous avons parlé au chapitre 2, Les objets en Python. Si nous ne fournissons pas ce paramètre, la docstring sera à la place copiée de la docstring pour le premier argument : la méthode getter

In [ ]:
class NorwegianBlue:    
  def __init__(self, name: str) -> None:        
    self._name = name        
    self._state: str    
    
  def _get_state(self) -> str:        
    print(f"Getting {self._name}'s State")        
    return self._state    
    
  def _set_state(self, state: str) -> None:        
    print(f"Setting {self._name}'s State to {state!r}")        
    self._state = state    
  
  def _del_state(self) -> None:        
    print(f"{self._name} is pushing up daisies!")        
    del self._state    
    
  silly = property(        
      _get_state, 
      _set_state, 
      _del_state,         
      "This is a silly property")

Notez que l'attribut state a un type hint, str, mais pas de valeur initiale. Il peut être supprimé et n'existe que pendant une partie de la vie d'un NorwegianBlue. 

Nous devons fournir un indice pour aider mypy à comprendre ce que le type devrait être. Mais nous n'attribuons pas de valeur par défaut car c'est le travail de la méthode setter

Si nous utilisons réellement une instance de cette classe, elle affiche en effet les chaînes correctes lorsque nous lui demandons 

In [ ]:
p = NorwegianBlue("Polly")

p.silly = "Pining for the fjords"

Setting Polly's State to 'Pining for the fjords'


In [ ]:
p.silly

Getting Polly's State


'Pining for the fjords'

In [ ]:
del p.silly

Polly is pushing up daisies!


## Décorateurs – une autre façon de créer des propriétés

Nous pouvons créer des propriétés à l'aide de décorateurs. Cela rend les définitions plus faciles à lire. Les décorateurs sont une caractéristique omniprésente de la syntaxe Python, avec une variété d'objectifs. Pour la plupart, les décorateurs modifient la définition de fonction qu'ils précèdent. Nous examinerons le modèle de conception du décorateur plus largement au guide 11, Modèles de conception communs. La fonction de propriété peut être utilisée avec la syntaxe du décorateur pour transformer une méthode get en un attribut de propriété, comme suit

In [ ]:
class NorwegianBlue_P:    
  def __init__(self, name: str) -> None:        
    self._name = name        
    self._state: str

  @property    
  def silly(self) -> str:        
    print(f"Getting {self._name}'s State")        
    return self._state

Cela applique la fonction de propriété en tant que décorateur à la fonction qui suit. 

Elle est équivalente à la syntaxe précédente silly = property(_get_state). La principale différence, du point de vue de la lisibilité, est que nous pouvons marquer la méthode stilly comme une propriété en haut de la méthode, au lieu d'après sa définition, où elle peut être facilement ignorée. 

Cela signifie également que nous n'avons pas à créer de méthodes privées avec des préfixes de soulignement juste pour définir une propriété. Pour aller plus loin, nous pouvons spécifier une fonction de définition pour la nouvelle propriété comme suit

In [ ]:
class NorwegianBlue_P:    
  def __init__(self, name: str) -> None:        
    self._name = name        
    self._state: str

  @property    
  def silly(self) -> str:        
    print(f"Getting {self._name}'s State")        
    return self._state
    
  @silly.setter    
  def silly(self, state: str) -> None:
    print(f"Setting {self._name}'s State to {state!r}")        
    self._state = state

Cette syntaxe, @silly.setter, semble étrange par rapport à @property, bien que l'intention doive être claire. Tout d'abord, nous décorons la méthode silly comme un getter. Ensuite, nous décorons une deuxième méthode avec exactement le même nom en appliquant l'attribut setter de la méthode silly décorée à l'origine ! 


Cela fonctionne car la fonction de propriété renvoie un objet ; cet objet a également son propre attribut setter, qui peut ensuite être appliqué comme décorateur à d'autres méthodes. L'utilisation du même nom pour les méthodes get et set permet de regrouper les multiples méthodes qui accèdent à un attribut commun. Nous pouvons également spécifier une fonction de suppression avec @silly.deleter. Voici à quoi cela ressemble


In [ ]:
class NorwegianBlue_P:    
  def __init__(self, name: str) -> None:        
    self._name = name        
    self._state: str

  @property    
  def silly(self) -> str:        
    print(f"Getting {self._name}'s State")        
    return self._state
      
  @silly.setter    
  def silly(self, state: str) -> None:
    print(f"Setting {self._name}'s State to {state!r}")        
    self._state = state

  @silly.deleter
  def silly(self) -> None:    
    print(f"{self._name} is pushing up daisies!")    
    del self._state